In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import json
import time
import sys
import os

weeks = 13

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)

def get_losers():
    
    print('Losers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Loss')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers("losers"))):
        print(alpaca_api.polygon.gainers_losers("losers")[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers("losers")[symbol].todaysChangePerc)+'%')
def get_gainers():
    print('Gainers'.center(45))
    print()
    print('Symbol'.ljust(10), 'Last'.ljust(11), 'Change'.ljust(11), 'Pct Gain')
    print("{:<30}".format('–' * 45))
    for symbol in range(len(alpaca_api.polygon.gainers_losers())):
        print(alpaca_api.polygon.gainers_losers()[symbol].ticker.ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].lastTrade['p']).ljust(10),
              '$' + str(alpaca_api.polygon.gainers_losers()[symbol].todaysChange).ljust(10),
              str(alpaca_api.polygon.gainers_losers()[symbol].todaysChangePerc)+'%')

if __name__ == '__main__':
    get_gainers()
    print()
    get_losers()
    print()
    print("Bulls".center(45))
    print("{:<30}".format('–' * 45))
    bulls = selector.bullish_candlesticks(64, 20)
    print()
    print("Bears".center(45))
    print("{:<30}".format('–' * 45))
    bears = selector.bearish_candlesticks(64, 20)

                   Gainers                   

Symbol     Last        Change      Pct Gain
–––––––––––––––––––––––––––––––––––––––––––––
SLS        $5.75       $5.614      4127.941%
TCON       $3.71       $3.332      882.521%
CBO        $19         $13.55      248.624%
SKYS       $1.55       $0.92       146.032%
  0         $0.2097     $7803471516837.872 74.169%
MICT       $0.2097     $0.089      74.169%
MICT       $1.07       $4705346816927.858 67.153%
NSPR.WS    $0.0079     $0.002      38.596%
CLVS       $5.9        $1.58       36.574%
UI         $174.6223   $44.632     34.335%
OTLKW      $0.172      $0.042      32.308%
ATISW      $0.0798     $0.018      29.967%
SGLBW      $0.0782     $0.018      29.9%
SIEN       $9.065      $2.065      29.5%
LEAF       $4.13       $0.94       29.467%
TRUE       $4.695      $1.045      28.63%
RAPT       $28.4       $6.13       27.526%
FLGT       $15.5       $3.18       25.812%
HHHHW      $0.0998     $0.02       24.439%
AAXN       $65.11      $12.7   